### Analysis of scenario 1 / Weekday

In [58]:
hour = '20'
# scenario = "Scenario1-Weekday/"
# scenario = "Scenario2-Weekday/"
# scenario = "Scenario1-Saturday/"
scenario = "Scenario2-Saturday/"

# filename = "/passengers_R_H"
# filename = "/passengers_M_H"
# filename = "/passengers_RS_H"
filename = "/passengers_MS_H"

path = "../../Results/" + scenario + hour + filename + hour + "_I"

df1 = pd.read_csv(path + "1.csv")
df2 = pd.read_csv(path + "2.csv")
df3 = pd.read_csv(path + "3.csv")
df4 = pd.read_csv(path + "4.csv")
df5 = pd.read_csv(path + "5.csv")
df6 = pd.read_csv(path + "6.csv")
df7 = pd.read_csv(path + "7.csv")
df8 = pd.read_csv(path + "8.csv")
df9 = pd.read_csv(path + "9.csv")
df10 = pd.read_csv(path + "10.csv")

dflist = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10]

In [59]:
from collections import defaultdict

origin = Counter()
destination = Counter()
od_pairs = Counter()

for i, df in enumerate(dflist):
    orig = Counter(dflist[i]['origin'].tolist())
    origin = origin + orig
    
    dest = Counter(dflist[i]['destination'].tolist())
    destination = destination + dest
    
#     od = Counter(zip(dflist[i]['origin'].tolist(), dflist[i]['destination'].tolist()))
#     od_pairs = od_pairs + od
    
origin = dict(origin)
destination = dict(destination)
# od_pairs = dict(od_pairs)

In [60]:
origin = {k: v / 10 for k, v in origin.items()}
destination = {k: v / 10 for k, v in destination.items()}
# od_pairs = {k: v / 10 for k, v in od_pairs.items()}

In [61]:
origin = pd.DataFrame.from_dict(origin, orient='index').sort_values(by = 0, ascending = False).reset_index()
destination = pd.DataFrame.from_dict(destination, orient='index').sort_values(by = 0, ascending = False).reset_index()
# od_pairs = pd.DataFrame.from_dict(od_pairs, orient='index').sort_values(by = 0, ascending = False).reset_index()

In [65]:
# from ast import literal_eval
# dfordest['station'] = [literal_eval(x) for x in dfordest['station']]
# dfordest = dfordest.merge(od_pairs, left_on='station', right_on='index', how='outer')

In [66]:
# dfordest.columns

Index(['station', 'sc1week', 'sc1weekend', 'sc2week', 'index', 0], dtype='object')

In [67]:
# dfordest.columns = ['station', 'sc1week', 'sc1weekend', 'sc2week', 'station2', 'sc2weekend']

In [68]:
# dfordest.station.fillna(dfordest.station2, inplace=True)

In [69]:
# del dfordest['station2']

In [24]:
import networkx as nx
import warnings
import datetime
import seaborn as sn
sn.set()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline
warnings.filterwarnings('ignore')

In [148]:
hour = "20"
dfor = pd.read_csv('../../Results/origin' + hour + '.csv')
dfdest = pd.read_csv('../../Results/destination'  + hour + '.csv')

In [149]:
origingvb = pd.read_csv('Data/originsaturday.csv')
destgvb = pd.read_csv('Data/destinationsaturday.csv')
origingvb = origingvb[origingvb['hour'] =="20:00 - 20:59"]
destgvb = destgvb[destgvb['hour'] =="20:00 - 20:59"]
destgvb = destgvb[['stop', 'mean']]
origingvb = origingvb[['stop', 'mean']]

In [150]:
dfor = dfor.merge(origingvb, how='outer', left_on='station', right_on='stop')

In [151]:
dfor.station.fillna(dfor.stop, inplace=True)

In [152]:
del dfor['stop']

In [153]:
dfor.columns = [['station', 'sc1week', 'sc1weekend', 'sc2week', 'sc2weekend', 'gvbweek', 'gvbweekend']]

In [154]:
dfor.sum(axis = 0, skipna = True) 

station       Station ZuidCentraal StationRokinDe PijpStatio...
sc1week                                                  4972.7
sc1weekend                                                 5811
sc2week                                                  4803.7
sc2weekend                                               5590.3
gvbweek                                                 6217.51
gvbweekend                                               7166.2
dtype: object

In [155]:
dfdest = dfdest.merge(destgvb, left_on='station', right_on='stop', how='outer')

In [156]:
dfdest.station.fillna(dfdest.stop, inplace=True)

In [157]:
del dfdest['stop']

In [158]:
dfdest.columns = [['station', 'sc1week', 'sc1weekend', 'sc2week', 'sc2weekend', 'gvbweek', 'gvbweekend']]

In [159]:
dfdest.sum(axis = 0, skipna = True) 

sc1week       4972.700000
sc1weekend    5811.000000
sc2week       4803.700000
sc2weekend    5590.300000
gvbweek       5854.814520
gvbweekend    6772.030303
dtype: float64

In [160]:
dfor.head()

,station,sc1week,sc1weekend,sc2week,sc2weekend,gvbweek,gvbweekend
0,Station Zuid,797.9,594.8,781.6,585.0,1089.727273,784.090909
1,Centraal Station,529.2,791.4,533.7,778.1,701.985732,1014.778788
2,Rokin,475.6,623.2,466.2,617.1,537.926768,718.909091
3,De Pijp,351.3,478.3,329.1,429.4,433.045455,593.651515
4,Station Bijlmer ArenA,209.3,222.9,207.2,230.0,273.518939,296.000000


In [161]:
dfdest.head()

,station,sc1week,sc1weekend,sc2week,sc2weekend,gvbweek,gvbweekend
0,Station Zuid,454.0,465.2,459.3,461.2,612.420455,581.590909
1,Centraal Station,445.8,496.3,413.8,463.7,561.402525,638.170707
2,Noord,419.0,465.4,414.2,450.8,489.295455,545.121212
3,De Pijp,362.9,467.3,362.8,457.9,419.428030,540.393939
4,Station Bijlmer ArenA,237.8,375.0,234.7,354.5,289.734848,447.196970


In [162]:
dfor.to_csv('../../Results/origin' + hour + '.csv', index=False)
dfdest.to_csv('../../Results/destination'  + hour +  '.csv', index=False)